In [1]:
%matplotlib inline

In [60]:
from IPython.display import Image
from copy import copy,deepcopy
from numpy import pi
import numpy as np
from qutip import *
from qutip.qip.operations import *
from qutip.qip.circuit import QubitCircuit, Gate
import time
from itertools import combinations


def user_gate1():
# S gate
    mat = np.array([[1.,0],[0., 1.j]])
    return Qobj(mat, dims=[[2], [2]])

def user_gate2():
# T gate
    mat = np.array([[1.,0],[0., 0.707+0.707j]])
    return Qobj(mat, dims=[[2], [2]])

In [70]:
def qu(gnum,n): # pick the unitary from the gate set
    
    if gnum<3*n+1: 
        if gnum%n==0:
            g_t= Gate("SNOT", int((gnum-1)/n))
        elif gnum%n==1:
            g_t= Gate("S", int((gnum-1)/n))
        else: 
            g_t= Gate("T",int((gnum-1)/n))
    elif gnum<3*n+n+1:
        #print((gnum-3*n)%n, (gnum-3*n+1)%n )
        g_t= Gate("CNOT",int((gnum-3*n)%n),int((gnum-3*n+1)%n)  )
    elif gnum<3*n+n*2+1:
        g_t= Gate("CNOT",int((gnum-3*n+1)%n), int((gnum-3*n)%n)  )
    return g_t

In [57]:
def quc(x,n): # generate the given quantum circuit
    qc= QubitCircuit(n)
    qc.user_gates = {"S": user_gate1,"T": user_gate2}
    for i in range(len(x)):
        if x[i]!=0:
            qc.add_gate(qu(x[i], n))
    prop = qc.propagators()
    return gate_sequence_product(prop).full()
    

In [53]:
#may be faster to apply inverse i.e. loop over numbers gmax**r in base gmax
# def list_circuits(gmax,r):# list the circuits
#     def list_c(*arrays): 
#         grid = np.meshgrid(*arrays)        
#         coord_list = [entry.ravel() for entry in grid]
#         points = np.vstack(coord_list).T
#         return points
#     aa = np.arange(gmax)
#     return list_c(*r*[np.arange(gmax)])

# below is time inefficient but memory efficient
# not optimal 28.8 vs 15.7

def base_conv(cirnum,gmax,r): 
    xx= np.zeros(r)
    for i in range(r): 
        xx[i]= cirnum%gmax
        cirnum//=gmax
    return xx



In [32]:
def bi(ii,n):
    bix = np.zeros(n)
    for i in range(n): 
        bix[i]= ii%2
        ii//=2
    return(bix)

def list_povm(n): 
    l_povms=[]
    for i in range(2**n):
        pv = np.zeros(2**n)
        b = bi(i,n)
        pv[0]= 1
        for j in range(len(b)): 
            if b[j]==1:
                pv[2**j:2**(j+1)]= pv[:2**j]
        l_povms.append(np.diag(pv))
    return(l_povms)


# below is time inefficient but memory efficient
#removed since, it is less optimal 4.67s vs 3.54s


# def gene_povm(gnp,n): 
#     pv = np.zeros(2**n)
#     b = bi(gnp,n)
#     pv[0]= 1
#     for j in range(len(b)): 
#         if b[j]==1:
#             pv[2**j:2**(j+1)]= pv[:2**j]
#     return(np.diag(pv)) 

            
                
                

In [35]:
def trqrho(q,rho):
    trq= 0
    trq+= np.sum(np.diag(q)*np.diag(rho))
    return(trq)

def tr(q): 
    return(np.sum(np.diag(q)))


In [17]:
a = combinations(range(5),2)
type(list(a)[5][0])

int

In [ ]:
# All the inputs are in this box 

# system specific data
n = 3 # number of qubits
gmax= 1+3*n+n*2 # number of unique gates in the universal gate set

# process specific
r = 4 # max complexity
eta = 0.7 # success rate? 
# define rho below this
x = np.random.rand(2**n,2**n)
rho= (x+x.transpose())/(2*np.trace(x))

In [33]:
povms = list_povm(n)


array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
circi=0
povmi=0
H=2**3
for cnum in range(1,gmax**r):
    #print(base_conv(cnum,gmax,r))
    U = quc(base_conv(cnum,gmax,r),n)
    #print(U)
    sigma = U@rho@U.conj() # use of einsum may make it faster? need to make this step faster
    for pov in range(len(povms)):
        povm = povms[pov]
        if trqrho(povm,sigma)>=eta and tr(povm)<H:
            circi = cnum
            povmi = pov
            H = tr(povm)
print("effective entropy for eta="+str(eta)+"and r="+str(r) +" is "+ str(np.log(H)))    

In [74]:
print(povms[povmi])
print(circi)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
873


In [64]:
qc= QubitCircuit(n)
qc.add_gate("SNOT", 1)
qc.add_gate("CNOT",1,2)

In [51]:
a = np.random.rand(8,8)
b = np.random.rand(8,8)
c = a@b@a.conj()
np.log(2.7)

0.9932517730102834

In [65]:
gnum=11
(gnum-3*n)%n, (gnum-3*n+1)%n

(2, 0)

5%2


In [22]:
props = qc.propagators()

In [46]:
U= gate_sequence_product(props).full()
U.conj()
# Q = np.random.rand(2**n,2**n)
# np.trace(U*Q)
    


array([[ 0.70710678-0.j,  0.        -0.j,  0.70710678-0.j,
         0.        -0.j,  0.        -0.j,  0.        -0.j,
         0.        -0.j,  0.        -0.j],
       [ 0.        -0.j,  0.70710678-0.j,  0.        -0.j,
         0.70710678-0.j,  0.        -0.j,  0.        -0.j,
         0.        -0.j,  0.        -0.j],
       [ 0.70710678-0.j,  0.        -0.j, -0.70710678-0.j,
         0.        -0.j,  0.        -0.j,  0.        -0.j,
         0.        -0.j,  0.        -0.j],
       [ 0.        -0.j,  0.70710678-0.j,  0.        -0.j,
        -0.70710678-0.j,  0.        -0.j,  0.        -0.j,
         0.        -0.j,  0.        -0.j],
       [ 0.        -0.j,  0.        -0.j,  0.        -0.j,
         0.        -0.j,  0.70710678-0.j,  0.        -0.j,
         0.70710678-0.j,  0.        -0.j],
       [ 0.        -0.j,  0.        -0.j,  0.        -0.j,
         0.        -0.j,  0.        -0.j,  0.70710678-0.j,
         0.        -0.j,  0.70710678-0.j],
       [ 0.        -0.j,  0.      

In [10]:
props = qc1.propagators()
props[0]

NameError: name 'qc1' is not defined

In [11]:
for i in range(10,13):
    print(i,(i-3*n)%n, (i-3*n+1)%n)

10 1 2
11 2 0
12 0 1


In [12]:
-1%3

2

In [13]:
a


NameError: name 'a' is not defined

In [15]:
a = np.arange(50)  # fake data
print(len(list_c(*r*[a])))


NameError: name 'list_c' is not defined

In [46]:
pv = np.random.rand(2**5)
j = 3
pv[2**j:2**(j+1)]= pv[:2**j]
print(pv[:2**5])

[0.49171691 0.98149052 0.31583407 0.50734355 0.71633766 0.6033593
 0.72866585 0.74982201 0.49171691 0.98149052 0.31583407 0.50734355
 0.71633766 0.6033593  0.72866585 0.74982201 0.57805718 0.4293471
 0.21136173 0.95261813 0.1583954  0.48094306 0.41077553 0.58631585
 0.70192571 0.74992607 0.46552065 0.14574783 0.05704509 0.02934502
 0.37451038 0.40412242]


In [37]:
print(np.arange(1,10))

[1 2 3 4 5 6 7 8 9]
